In [1]:
import keras
from keras.models import Model
from keras.layers import Dense, Dropout, Activation, Flatten, Input, Concatenate
from keras.layers import Conv2D, MaxPooling2D
from keras.layers.merge import add

from keras import regularizers

import numpy as np

Using TensorFlow backend.


# Model Definition

## Encoder
The role of this model is to create feature vector from raw images.

Borrow from https://github.com/uzh-rpg/rpg_public_dronet

Note:
    Layer naming convention: `layer_name(str) + stage(int) + block(str)`

### Define a convolutional block

In [2]:
def convolutional_block(X, num_filters, shape_filters, strides, stage):
    """
    Implementation of convolutional block in Residual network
    
    Input:
        X (tensor): input tensor of shape (m, n_H_prev, n_W_prev, n_C_prev)
        num_filters (list of 3 ints): list of number of filters
        shape_filters (list of 3 ints): list of filters' shape
        strides (list of 3 ints): list of strides
        stage (int): stage of this convolutional block in the whole ResNet
        
    Output:
        tensor of shape (m, n_H, n_W, n_C)
    """
    
    # retrieve filters shape from filters
    n1, n2, n3 = num_filters
    f1, f2, f3 = shape_filters
    
    # retrieve strides from strides
    s1, s2, s3 = strides
    
    # create name
    bn_name_base = 'bn_' + str(stage) + '_'
    conv_name_base = 'conv_' + str(stage) + '_'
    
    # save value of X
    X_shorcut = X
    
    # First component of the main path
    X = keras.layers.normalization.BatchNormalization(name=bn_name_base + 'a')(X)
    X = Activation('relu')(X)
    X = Conv2D(n1, (f1, f1), strides=[s1, s1], padding='same',
               name=conv_name_base + 'a',
               kernel_initializer='he_normal',
               kernel_regularizer=regularizers.l2(1e-4))(X)
    
    # Second component of the main path
    X = keras.layers.normalization.BatchNormalization(name=bn_name_base + 'b')(X)
    X = Activation('relu')(X)
    X = Conv2D(n2, (f2, f2), strides=[s2, s2], padding='same',
               name=conv_name_base + 'b',
               kernel_initializer='he_normal',
               kernel_regularizer=regularizers.l2(1e-4))(X)
    
    # Short-cut
    X_shorcut = Conv2D(n3, (f3, f3), strides=[s3, s3], padding='same', name=conv_name_base + 'c')(X_shorcut)
    
    X = add([X, X_shorcut])
    
    return X


### Assemble 3 convolutional block to make a ResNet 

In [3]:
def resnet8_clean(input_shape):
    """
    Define encoder architecture as ResNet8
    
    Input:
        input_shape (list of ints): shape of input image [n_H, n_W, n_C]
        
    Output:
        model: a Model instance
    """
    
    # Input
    X_input = Input(shape=input_shape)
    
    # Apply 1st convolution & max pooling on input
    X = Conv2D(32, (5, 5), strides=[2,2], padding='same', name='conv_0')(X_input)
    X = MaxPooling2D(pool_size=(3, 3), strides=[2,2])(X) 
    
    # First convolutional block
    X = convolutional_block(X, [32, 32, 32], [3, 3, 1], [2, 1, 2], stage=1)
    
    # Second convolutional block
    X = convolutional_block(X, [64, 64, 64], [3, 3, 1], [2, 1, 2], stage=2)
    
    # Third convolutional block
    X = convolutional_block(X, [128, 128, 128], [3, 3, 1], [2, 1, 2], stage=3)
    
    # Output layer
    X = Flatten()(X)
    X = Activation('relu')(X)
    
    # Define model
    model = Model(inputs=[X_input], outputs=[X])
    print(model.summary())
    
    return model

### Create model & load weights

In [4]:
encoder = resnet8_clean([200, 200, 1])
encoder.load_weights('./model/named_resnet8_best_weights.h5', by_name=True)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 200, 200, 1)  0                                            
__________________________________________________________________________________________________
conv_0 (Conv2D)                 (None, 100, 100, 32) 832         input_1[0][0]                    
__________________________________________________________________________________________________
max_pooling2d_1 (MaxPooling2D)  (None, 49, 49, 32)   0           conv_0[0][0]                     
__

In [48]:
for l in encoder.layers:
    l.trainable = False

## Decoder

This model decode the feature vector created by the Encoder to produce the prediction for the `PLANNING_HORIZON` (in form of classes of steering angles) 

In [45]:
def fully_connected(input_shape, num_classes, planning_horizon):
    """
    Define a network of 3 dense layer
    
    Input:
        input_shape (list of ints)
        num_classes (int)
        
    Output:
        Model instance
    """
    X_input = Input(shape=input_shape)
    
    X = Dropout(0.5)(X_input)
     
    X = Dense(500, activation=None)(X)
    
    y = [Dense(num_classes, activation='softmax')(X) for i in range(planning_horizon)]
    
    y = Concatenate()(y)
    
    model = Model(inputs=[X_input], outputs=[y])
    print(model.summary())
    
    return model
    

In [46]:
decoder = fully_connected([3, 6272], 400, 25)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_10 (InputLayer)           (None, 3, 6272)      0                                            
__________________________________________________________________________________________________
dropout_10 (Dropout)            (None, 3, 6272)      0           input_10[0][0]                   
__________________________________________________________________________________________________
dense_44 (Dense)                (None, 3, 500)       3136500     dropout_10[0][0]                 
__________________________________________________________________________________________________
dense_45 (Dense)                (None, 3, 400)       200400      dense_44[0][0]                   
__________________________________________________________________________________________________
dense_46 (